In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import chromedriver_autoinstaller
import time
from selenium.webdriver.common.by import By
import re
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pickle

In [4]:
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--lang=ko_KR')
    chrome_options.add_argument('--window-size=1920x1080')

    # headless임을 숨기기 위해서
    # headless인 경우 Cloudflare 서비스가 동작한다.
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    
    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
    try:
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')   
    except:
        chromedriver_autoinstaller.install(True)
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')

    driver.implicitly_wait(10)
    
    return driver

In [12]:
# 드라이버 호출 -> 실행
driver = set_chrome_driver()

C:\Users\LSH\AppData\Local\Temp/ipykernel_17640/2544243862.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')


In [6]:
# base
driver.implicitly_wait(1)
driver.get('https://store.steampowered.com/')
time.sleep(1)

In [7]:
with open('review.pk' , 'rb') as f:
    eng_review = pickle.load(f)

In [8]:
game_name = eng_review['game_name'].tolist()

In [9]:
error = []
engreview_eval = []

In [10]:
def find_game(game, string):
    game = game.lower()
    string = string.lower()

    game_resub = re.sub('[^a-z0-9]','',game)
    string_resub = re.sub('[^a-z0-9I]','',string)
    
    if game_resub == string_resub:
        return True

In [14]:
not_in_list = ['people found this review', 'early access review', 'recommended', 'hrs on record'
                              , 'posted: ', 'products in account', 'product received for free', 
                              'person found this review funny', 'no one has rated this review as helpful yet']
num = 959 #282
while num < 1001:
    driver.get('https://store.steampowered.com/')
    print('{}번째 크롤링'.format(num))
    try:
        name = game_name[num-1]
        print('{} 크롤링 중'.format(name))
        driver.find_element("xpath", '//*[@id="store_nav_search_term"]').send_keys(name)
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="store_search_link"]/img').click()
        time.sleep(1)

        for i in range(1,20):
            string = driver.find_element("xpath",'//*[@id="search_resultsRows"]/a[{}]/div[2]/div[1]/span'.format(i)).text
            if find_game(name, string):
                driver.find_element("xpath",'//*[@id="search_resultsRows"]/a[{}]/div[2]/div[1]/span'.format(i)).click()
                break
        time.sleep(1)
    except:
        name = re.sub("[^A-Za-z0-9']+",' ',name)
        print('{} 크롤링 중'.format(name))
        driver.find_element("xpath", '//*[@id="store_nav_search_term"]').send_keys(name)
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="store_search_link"]/img').click()
        time.sleep(1)

        for i in range(1,20):
            string = driver.find_element("xpath",'//*[@id="search_resultsRows"]/a[{}]/div[2]/div[1]/span'.format(i)).text
            if find_game(name, string):
                driver.find_element("xpath",'//*[@id="search_resultsRows"]/a[{}]/div[2]/div[1]/span'.format(i)).click()
                break
        time.sleep(1)
    

    
    # 19+
    try:
        driver.find_element("xpath", '//*[@id="app_agegate"]/div[1]/div[3]')
        driver.find_element("xpath", '//*[@id="ageYear"]').send_keys('1997')
        driver.find_element(By.XPATH, '//*[@id="view_product_page_btn"]').click()
        time.sleep(1)
    except Exception as e:
        print(e)
        
    # get review
    # review = [ { review : good / not good }, ... ]
    # order by 'helpful' desc
    # if review count >= 100: stop
    review_count = 0
    # scroll event
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    try:
        # click all review
        driver.find_element(By.XPATH, '//*[@id="ViewAllReviewssummary"]/a').click()
        time.sleep(2)
    
        # community error
        try:
            driver.find_element("xpath", '//*[@id="ViewAllForApp"]').click()
            driver.find_element("xpath", '//*[@id="age_gate_btn_continue"]/span').click()
            time.sleep(1)
        except Exception as e:
            print(e)
        
        dict_review = {}
        try:
            # page cycle
            for page in range(1, 100):
    
                for _ in range(2):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)

                review_page = driver.find_element("xpath", '//*[@id="page{}"]'.format(page)).text
                pages = review_page.split('\n')

                evaluated = []
                for senten in pages:
                    sen = senten.lower()
                    if sen == 'recommended':
                        evaluated.append('1')
                    if sen == 'not recommended':
                        evaluated.append('0')


                review_lists = []
                for senten in pages:
                    sen = senten.lower()
                    if 'people found this review' not in sen:
                        if 'early access review' not in sen:
                            if 'recommended' not in sen:
                                if 'hrs on record' not in sen:
                                    if 'posted: ' not in sen:
                                        if 'products in account' not in sen:
                                            if 'product received for free' not in sen:
                                                if 'person found this review funny' not in sen:
                                                    if 'no one has rated this review as helpful yet' not in sen:
                                                        review_lists.append(sen)

                preprocessed_reviews = []
                review_split = []

                for sen in review_lists:
                    if sen.isdigit():
                        review_split = " ".join(review_split)
                        preprocessed_reviews.append(review_split)
                        review_split = []
                    else:
                        review_split.append(sen)

                for sen, evaluate in zip(preprocessed_reviews, evaluated):
                    dict_review[sen] = int(evaluate)

        except:
            print('Page_rangeout')
            pass
    
        engreview_eval.append(dict_review)
    except:
        print('no review')
        engreview_eval.append('No review')
    
    if num % 50 == 0:
        with open('2engreview_eval{}.pk'.format(num) , 'wb') as f:
            pickle.dump(engreview_eval, f)
            

    time.sleep(1)

    num +=1

959번째 크롤링
Fallen girl - Black rose and the fire of desire 크롤링 중
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app_agegate"]/div[1]/div[3]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Page_rangeou

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ViewAllForApp"]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

965번째 크롤링
Warhammer: Chaosbane 크롤링 중
Message: no such element: Unable to locate element:

970번째 크롤링
DARK SOULS™ II 크롤링 중
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app_agegate"]/div[1]/div[3]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Message: no such element: Unable to locate el

Page_rangeout
975번째 크롤링
Gunvolt Chronicles: Luminous Avenger iX 크롤링 중
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app_agegate"]/div[1]/div[3]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Messag

980번째 크롤링
Onimusha: Warlords 크롤링 중
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app_agegate"]/div[1]/div[3]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Page_rangeout
981번째 크롤링
Saimin Gakushū: S

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ViewAllForApp"]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

986번째 크롤링
The Ramen Sensei 크롤링 중
Message: no such element: Unable to locate element: {"m

Page_rangeout
990번째 크롤링
Counter Fight: Samurai Edition 크롤링 중
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app_agegate"]/div[1]/div[3]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Message: no suc

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ViewAllForApp"]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Page_rangeout
996번째 크롤링
Close Combat: The Bloody First 크롤링 중
Message: no such element: U

In [15]:
len(engreview_eval)

1000